In [5]:
import os

In [6]:
%pwd

'd:\\A_Category\\iNeuron\\End-To-End-NLP-Project-News-Article-Sorting\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'd:\\A_Category\\iNeuron\\End-To-End-NLP-Project-News-Article-Sorting'

In [36]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModeTrainerConfig:
  root_dir: Path
  data_path: Path
  model_ckpt: Path
  output_dir: Path
  learning_rate: float
  per_device_train_batch_size: int
  per_device_eval_batch_size: int
  num_train_epochs: int
  weight_decay: float
  eval_steps: int
  evaluation_strategy: str
  save_strategy: str
  load_best_model_at_end: bool

In [37]:
from ArticleSorting.constants import *
from ArticleSorting.utils.common import read_yaml, create_directories

In [38]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModeTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModeTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            output_dir = params.output_dir,
            learning_rate  = params.learning_rate,
            per_device_train_batch_size  = params.per_device_train_batch_size,
            per_device_eval_batch_size  = params.per_device_eval_batch_size,
            num_train_epochs  = params.num_train_epochs,
            weight_decay= params.weight_decay,
            eval_steps= params.eval_steps,
            evaluation_strategy= params.evaluation_strategy,
            save_strategy = params.save_strategy,
            load_best_model_at_end= params.load_best_model_at_end
        )

        return model_trainer_config


In [47]:
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset, load_from_disk, get_dataset_split_names

In [48]:
class ModeTrainer:
    def __init__(self, config: ModeTrainerConfig) :
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(device)
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_bert = AutoModelForSequenceClassification.from_pretrained(self.config.model_ckpt).to(device)
        
        # Loading data
        dataset = load_from_disk(self.config.data_path)
        # Spliting the dataset
        get_dataset_split_names(dataset)['train_dataset', 'test_dataset']

        # Training dataset
        #train_dataset = dataset.sample(frac=0.8, random_state=42)
        #print (len(train_dataset))
        # Testing dataset
        #test_dataset = dataset.drop(train_dataset.index)
        #print (len(test_dataset))

        # Change the format to PyTorch tensors
        train_dataset.set_format("torch")
        test_dataset.set_format("torch")

        # DataLoader
        train_dataloader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=4)
        eval_dataloader = DataLoader(dataset=test_dataset, batch_size=4)

In [49]:
try:
    config = ConfigurationManager()
    model_trainer_comfig = config.get_model_trainer_config()
    model_trainer = ModeTrainer(config= model_trainer_comfig)
    model_trainer.train()

except Exception as e:
    raise e

[2023-10-28 11:44:48,096:  INFO: common: yaml file:config\config.yaml loaded successfully]
[2023-10-28 11:44:48,101:  INFO: common: yaml file:params.yaml loaded successfully]
[2023-10-28 11:44:48,103:  INFO: common: created directory at : artifacts]
[2023-10-28 11:44:48,107:  INFO: common: created directory at : artifacts/model_trainer]
cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'Dataset' object has no attribute 'replace'